<a href="https://colab.research.google.com/github/Chidiebere-Ogbuchi/TSP_CausalDiscovery/blob/main/Preproc_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import numpy as np
import os
import gc
import glob
import zipfile
import shutil


gc.collect()

97

In [104]:
locations = ["entrance", "hall1", "hall2", "hall3", "shop", "rest", "edge", "metaverse"]

def map_location(location):
    location_lower = location.lower()  # Convert location to lowercase
    if location_lower == "entrance":
        return 2
    elif location_lower == "hall1":
        return 1
    elif location_lower == "hall2":
        return 3
    elif location_lower == "hall3":
        return 4
    elif location_lower == "shop":
        return 5
    elif location_lower == "rest":
        return 6
    elif location_lower == "edge":
        return 7
    elif location_lower == "metaverse":
        return 8
    else:
        return None  # Return None for other cases


def assign_value(observation):
    split = observation.split('-')
    last_part = split[0]
    if last_part == 'smartlighting':
        return 1
    elif last_part == 'visitorguiding':
        return 2
    elif last_part == 'maintenance':
        return 3
    elif last_part == 'security':
        return 4
    else:
        return 5  # Return None for cases not specified





In [105]:
# Define the directory path where you want to search for zip files
directory_path = "./"

# Use glob to find all zip files in the directory
zip_files = glob.glob(os.path.join(directory_path, "*.zip"))

# Print the list of zip files
print("Zip files found in the directory:")
for zip_file in zip_files:
    print(zip_file)

# Save the result in a list
zip_file_list = zip_files

# Print the list
print("Zip files list:")
print(zip_file_list)


Zip files found in the directory:
./Monday_02_3.0.zip
./Sunday_01_7.0.zip
./Sunday_01_3.0.zip
./Monday_01_7.0.zip
./Monday_02_7.0.zip
./Sunday_02_3.0.zip
./Sunday_02_7.0.zip
./Monday_01_3.0.zip
Zip files list:
['./Monday_02_3.0.zip', './Sunday_01_7.0.zip', './Sunday_01_3.0.zip', './Monday_01_7.0.zip', './Monday_02_7.0.zip', './Sunday_02_3.0.zip', './Sunday_02_7.0.zip', './Monday_01_3.0.zip']


In [106]:
# Define the directory path where you want to search for zip files
directory_path = "./"

# Use glob to find all zip files in the directory
zip_files = glob.glob(os.path.join(directory_path, "*.zip"))

# Loop through each zip file
for zip_file_path in zip_files:
    # Get the name of the zip file without extension
    zip_file_name = os.path.splitext(os.path.basename(zip_file_path))[0]

    # Specify the directory where you want to extract the contents
    extracted_dir_path = os.path.join('extracted', zip_file_name)

    # Check if the directory exists and is not empty
    if os.path.exists(extracted_dir_path) and os.listdir(extracted_dir_path):
        print(f"Directory '{extracted_dir_path}' is not empty. Clearing contents...")
        # Forcefully remove the directory and its contents
        shutil.rmtree(extracted_dir_path)
        print("Directory contents cleared.")

    # Create the directory if it doesn't exist
    os.makedirs(extracted_dir_path, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir_path)

    print(f"Extraction complete for '{zip_file_name}'.")

    # Define the list of days of the week
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    # Define the base directory
    base_directory = "/content/extracted/"

    # Check if any folder in the base directory contains any day of the week
    folder_found = False
    for folder_name in os.listdir(base_directory):
        for day in days_of_week:
            if day in folder_name:
                full_path = os.path.join(base_directory, folder_name)
                print(f"Folder '{full_path}' contains '{day}'.")
                folder_found = True
                break  # Stop searching for the day once found
        if folder_found:
            break  # Stop searching if any folder is found

    if not folder_found:
        print("No folder contains any day of the week.")

    # Split the full path to get the folder name
    _, folder_name = os.path.split(full_path)

    day, week, bw = folder_name.split("_")

    # Convert bandwidth to float
    week = int(week)
    bw = float(bw)

    print("Day:", day)
    print("Week:", week)
    print("Bandwidth:", bw)

    day_week = "_".join(folder_name.split("_")[:2])
    print(day_week)

    # Define the full path to the CSV file
    csv_file_path = full_path + "/" + day_week + "/visitor_maps.csv"

    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Add new columns "day" and "week" to the DataFrame and populate them
    df['day'] = day
    df['week'] = week
    df['Bandwidth'] = bw

    bw = int(bw)

    # Define a dictionary to map time periods to numerical values
    time_period_mapping = {
        "8am to 11am": 1,
        "11am to 3pm": 2,
        "3pm to 7pm": 3
    }

    # Replace the values in the "Time Period" column with numerical values
    df['Time Period'] = df['Time Period'].replace(time_period_mapping)

    # Define a dictionary to map days of the week to numbers
    day_number_mapping = {day: i+1 for i, day in enumerate(days_of_week)}

    # Replace the values in the "day" column with their corresponding numbers
    df['day'] = df['day'].replace(day_number_mapping)

    # Define the full path to the CSV file
    csv_devices = full_path + "/" + day_week + "/devices.csv"

    # Load the CSV file into a DataFrame
    df_2 = pd.read_csv(csv_devices)

    # Merge the dataframes on the "Location" column
    merged_df = pd.merge(df, df_2, on="Location", how="inner")


    # Initialize an empty dictionary to store the mappings
    nbpeople_mappings_read = {}

    # Iterate over the rows of the DataFrame
    for index, row in merged_df.iterrows():
        # Get the location and visitor count for the current row
        location = row['Location']
        visitors = row['Visitors']

        # Determine the visitor label based on the time period
        if row['Time Period'] == 1:
            visitor_label = 'visitor_A'
        elif row['Time Period'] == 2:
            visitor_label = 'visitor_B'
        else:
            visitor_label = 'visitor_C'

        # Update the mappings dictionary
        if location not in nbpeople_mappings_read:
            nbpeople_mappings_read[location] = {}
        nbpeople_mappings_read[location][visitor_label] = visitors


    # List of folder names
    folders = ["entrance", "hall1", "hall2", "hall3", "shop", "rest", "edge", "metaverse"]

    # fpath = full_path + "/"

    # Iterate through each folder
    for folder_name in folders:
        # Define the path to the folder
        folder_path = os.path.join(full_path, folder_name)

        # Check if the folder exists
        if os.path.exists(folder_path):
            # Define the path to the results.csv file
            results_file_path = os.path.join(folder_path, 'results.csv')

            # Check if the results.csv file exists
            if os.path.exists(results_file_path):
                # Define the new filename
                new_filename = f"results_{folder_name}_{bw}_pr.csv"

                # Rename the results.csv file to the new filename
                os.rename(results_file_path, os.path.join(folder_path, new_filename))
                print(f"Renamed results.csv in '{folder_name}' folder to '{new_filename}'")
            else:
                print(f"No results.csv file found in '{folder_name}' folder.")
        else:
            print(f"'{folder_name}' folder does not exist.")

    period = 20

    # Define column names
    columns = ["timestamp", "sensorid", "observation", "location", "bandwidth", "payload", "responsetime"]

    # Iterate through each folder
    for folder_name, mappings in nbpeople_mappings_read.items():
        # Define the path to the result.csv file
        result_file_path = os.path.join(full_path, folder_name, f'results_{folder_name}_{bw}_pr.csv')

        # Check if the file exists
        if os.path.exists(result_file_path):
            # Read the CSV file into a DataFrame with specified column names
            df = pd.read_csv(result_file_path, names=columns)

            # Convert 'timestamp' column to datetime type
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # Sort the DataFrame by 'timestamp'
            df.sort_values(by='timestamp', inplace=True)

            # Define time intervals
            interval_duration = pd.Timedelta(seconds=period)

            # Define start time for each interval
            start_time_A = df['timestamp'].min()
            start_time_B = start_time_A + interval_duration
            start_time_C = start_time_B + interval_duration

            # Create 'Time-period' column based on time intervals
            df['Time-period'] = 3  # Default value for Time-period 3
            df.loc[(df['timestamp'] >= start_time_A) & (df['timestamp'] < start_time_B), 'Time-period'] = 1
            df.loc[(df['timestamp'] >= start_time_B) & (df['timestamp'] < start_time_C), 'Time-period'] = 2

            # Assign visitor counts based on mappings
            df['visitors'] = mappings['visitor_C']  # Default value for visitors
            df.loc[(df['timestamp'] >= start_time_A) & (df['timestamp'] < start_time_B), 'visitors'] = mappings['visitor_A']
            df.loc[(df['timestamp'] >= start_time_B) & (df['timestamp'] < start_time_C), 'visitors'] = mappings['visitor_B']

            # Assuming 'sensorid' is an important categorical variable
            df = df.groupby('sensorid', group_keys=False).apply(lambda x: x.sample(frac=0.75))

            # Save the modified DataFrame to a new CSV file
            output_file_path = f'results_{folder_name}_{bw}_mod.csv'
            df.to_csv(output_file_path, index=False)
            print(f"Processed file saved to {output_file_path}")
        else:
            print(f"File not found: {result_file_path}")

    # Get a list of all files ending with '_mod.csv'
    output_files = glob.glob(f"*{bw}_mod.csv")

    # Check if there are any files to concatenate
    if output_files:
        # Read each CSV file and concatenate into a single DataFrame
        combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)

        # Save the combined DataFrame to a new CSV file
        # Create the directory if it doesn't exist
        output_directory = './data/'
        os.makedirs(output_directory, exist_ok=True)

        # Convert "Location" column in df_1 to lowercase
        combined_df['location'] = combined_df['location'].str.lower()

        # Rename columns in df_2 to match the column names in df_1
        merged_df = merged_df.rename(columns={'Time Period': 'Time-period', 'Devices Sum': 'Devices_Sum',
                                    'Virtualsensors Sum': 'Virtualsensors_Sum', 'Applications Sum': 'Applications_Sum', 'Location': 'location'})

        # Merge the dataframes based on "Time-period" and "Location"
        final_df = pd.merge(combined_df, merged_df, on=['Time-period', 'location'], how='inner')

        # Apply mapping functions
        final_df['location_encoded'] = final_df['location'].apply(map_location)
        final_df['sensor'] = final_df['sensorid'].apply(assign_value)

        final_df = final_df.drop(['timestamp', 'bandwidth', 'payload', 'observation', 'sensorid', 'location', 'visitors'], axis=1)

        # Print the merged dataframe
        final_df.head()


        combined_csv_path = f'./data/{day_week}_{bw}_cres.csv'
        final_df.to_csv(combined_csv_path, index=False)

        print(f'Combined CSV file saved at: {combined_csv_path}')

        # Delete individual CSV files
        for file in output_files:
            os.remove(file)
        print('Individual CSV files deleted.')
    else:
        print('No files ending with "_mod.csv" found in the root directory.')

    !rm -r /content/extracted

    gc.collect()

Extraction complete for 'Monday_02_3.0'.
Folder '/content/extracted/Monday_02_3.0' contains 'Monday'.
Day: Monday
Week: 2
Bandwidth: 3.0
Monday_02
Renamed results.csv in 'entrance' folder to 'results_entrance_3_pr.csv'
Renamed results.csv in 'hall1' folder to 'results_hall1_3_pr.csv'
Renamed results.csv in 'hall2' folder to 'results_hall2_3_pr.csv'
Renamed results.csv in 'hall3' folder to 'results_hall3_3_pr.csv'
Renamed results.csv in 'shop' folder to 'results_shop_3_pr.csv'
Renamed results.csv in 'rest' folder to 'results_rest_3_pr.csv'
Renamed results.csv in 'edge' folder to 'results_edge_3_pr.csv'
Renamed results.csv in 'metaverse' folder to 'results_metaverse_3_pr.csv'
Processed file saved to results_entrance_3_mod.csv
Processed file saved to results_hall1_3_mod.csv
Processed file saved to results_hall2_3_mod.csv
Processed file saved to results_hall3_3_mod.csv
Processed file saved to results_shop_3_mod.csv
Processed file saved to results_rest_3_mod.csv
Processed file saved to resu

<ipython-input-106-ba8e4857e173>:213: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)


Combined CSV file saved at: ./data/Sunday_01_7_cres.csv
Individual CSV files deleted.
Extraction complete for 'Sunday_01_3.0'.
Folder '/content/extracted/Sunday_01_3.0' contains 'Sunday'.
Day: Sunday
Week: 1
Bandwidth: 3.0
Sunday_01
Renamed results.csv in 'entrance' folder to 'results_entrance_3_pr.csv'
Renamed results.csv in 'hall1' folder to 'results_hall1_3_pr.csv'
Renamed results.csv in 'hall2' folder to 'results_hall2_3_pr.csv'
Renamed results.csv in 'hall3' folder to 'results_hall3_3_pr.csv'
Renamed results.csv in 'shop' folder to 'results_shop_3_pr.csv'
Renamed results.csv in 'rest' folder to 'results_rest_3_pr.csv'
Renamed results.csv in 'edge' folder to 'results_edge_3_pr.csv'
Renamed results.csv in 'metaverse' folder to 'results_metaverse_3_pr.csv'


<ipython-input-106-ba8e4857e173>:171: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(result_file_path, names=columns)


Processed file saved to results_entrance_3_mod.csv
Processed file saved to results_hall1_3_mod.csv
Processed file saved to results_hall2_3_mod.csv
Processed file saved to results_hall3_3_mod.csv
Processed file saved to results_shop_3_mod.csv


...

Processed file saved to results_rest_3_mod.csv
Processed file saved to results_edge_3_mod.csv
Processed file saved to results_metaverse_3_mod.csv


<ipython-input-106-ba8e4857e173>:213: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)
<ipython-input-106-ba8e4857e173>:213: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)


Combined CSV file saved at: ./data/Sunday_01_3_cres.csv
Individual CSV files deleted.
Extraction complete for 'Monday_01_7.0'.
Folder '/content/extracted/Monday_01_7.0' contains 'Monday'.
Day: Monday
Week: 1
Bandwidth: 7.0
Monday_01
Renamed results.csv in 'entrance' folder to 'results_entrance_7_pr.csv'
Renamed results.csv in 'hall1' folder to 'results_hall1_7_pr.csv'
Renamed results.csv in 'hall2' folder to 'results_hall2_7_pr.csv'
Renamed results.csv in 'hall3' folder to 'results_hall3_7_pr.csv'
Renamed results.csv in 'shop' folder to 'results_shop_7_pr.csv'
Renamed results.csv in 'rest' folder to 'results_rest_7_pr.csv'
Renamed results.csv in 'edge' folder to 'results_edge_7_pr.csv'
Renamed results.csv in 'metaverse' folder to 'results_metaverse_7_pr.csv'
Processed file saved to results_entrance_7_mod.csv
Processed file saved to results_hall1_7_mod.csv
Processed file saved to results_hall2_7_mod.csv
Processed file saved to results_hall3_7_mod.csv
Processed file saved to results_shop

<ipython-input-106-ba8e4857e173>:213: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)


Combined CSV file saved at: ./data/Monday_01_3_cres.csv
Individual CSV files deleted.


In [107]:

# Get a list of all files ending with 'combined_results.csv'
output_files = glob.glob('./data/*_cres.csv')

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each output file
for output in output_files:
    df = pd.read_csv(output)
    dfs.append(df)

# Concatenate all DataFrames in the list along rows
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_directory = './data/'
combined_csv_path = f'./data/final_results.csv'
combined_df.to_csv(combined_csv_path, index=False)

print(f'Combined CSV file saved at: {combined_csv_path}')


Combined CSV file saved at: ./data/final_results.csv


In [109]:
# Define the path to the CSV file
csv_file_path = '/content/data/final_results.csv'

# Define the directory path for the zip file
zip_directory = '/content/final/'

# Create the directory if it doesn't exist
os.makedirs(zip_directory, exist_ok=True)

# Define the path to the zip file
zip_file_path = os.path.join(zip_directory, 'final_results.zip')

# Create a zip file and add the CSV file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(csv_file_path, arcname=os.path.basename(csv_file_path))

print(f'CSV file "{csv_file_path}" zipped successfully to "{zip_file_path}"')


CSV file "/content/data/final_results.csv" zipped successfully to "/content/final/final_results.zip"
